In [2]:
# Set project folder path so we can access all the files
import sys
import os

py_file_location = "/content/drive/MyDrive/Drug Response Prediction"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
# Install necessary dependency
!pip install -r requirements.txt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Install libraries
import warnings
import numpy as np
import pandas as pd
from os import walk
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

from collections import OrderedDict
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.decomposition import PCA
from sklearn.metrics import explained_variance_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout, Reshape, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import layers, models
from skimage.transform import resize
from tensorflow.keras import layers, models
from Feature_extraction import *
from Data_processing import *
from Dataset import *

warnings.simplefilter(action='ignore', category=FutureWarning)

# Load Data

In [5]:
result_df = merge_gene_expression_cn()

In [6]:
result_df

,cell_line,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,DERPC,AC055839.2,AP001453.3,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,NPBWR1,CDR1
0,ACH-000002,0.362532,0.274371,3.343203,1.125238,2.065523,2.576643,0.587594,2.614776,2.558101,...,2.636606,0.382942,2.148868,1.528380,0.671607,0.578567,1.263599,0.468970,0.483255,0.274371
1,ACH-000004,1.302372,0.214211,3.138665,1.800551,2.521042,1.110136,3.039840,2.548120,2.988215,...,2.466102,0.972584,1.917304,0.830707,0.815626,0.779967,1.734648,0.688572,0.639767,0.214211
2,ACH-000005,1.462814,0.237043,3.357664,2.019082,3.204829,0.762406,3.434712,2.629391,3.315447,...,2.790420,0.537556,1.687354,1.001156,1.228753,0.765769,1.766875,0.999276,0.660241,0.236395
3,ACH-000006,0.321167,0.272362,3.609158,1.939108,2.653499,2.447577,2.518025,3.351510,2.977455,...,1.918512,0.444875,1.906332,1.028573,0.661440,0.794750,1.778459,1.015335,0.511737,0.272362
4,ACH-000007,1.951300,0.276052,3.463491,2.201603,2.495457,0.500130,0.648881,3.296798,2.965959,...,1.615766,0.492227,2.172098,1.091848,0.481683,0.558021,1.734446,0.534023,0.478507,0.276052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,ACH-002780,1.505289,0.844955,3.440203,1.912507,2.851035,0.608485,0.817109,0.647047,2.224928,...,2.203146,1.878552,1.213025,0.926369,1.013786,0.594238,1.431874,1.551873,0.804401,1.184213
1068,ACH-002800,3.025578,0.378819,4.088177,1.798679,2.028179,0.440857,2.725448,3.326211,2.544003,...,2.354491,2.451189,1.533475,0.933998,0.833766,1.027447,1.592989,1.105811,1.072538,0.378819
1069,ACH-002801,2.024998,0.324729,3.908770,1.781532,2.168537,0.555371,3.722398,3.385181,3.047340,...,2.823719,1.576182,1.773806,1.066807,1.102800,1.040029,2.068667,0.566768,0.342114,0.457343
1070,ACH-002806,2.149806,0.354925,3.792027,0.996675,1.663626,0.513431,0.407190,0.698912,2.444723,...,2.435597,1.698297,1.614915,1.054681,0.920357,1.085026,2.005976,0.599049,0.768664,0.360095


In [7]:
drug_viability = load_drug_viability_data()

In [8]:
# Find common cell lines
set1 = set(result_df.cell_line)
set2 = set(drug_viability.cell_line)

common_cell_line = set2.intersection(set1)
len(common_cell_line)

467

In [9]:
result_df_filter = result_df[result_df['cell_line'].isin(common_cell_line)]

In [10]:
result_df_filter

,cell_line,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,DERPC,AC055839.2,AP001453.3,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,NPBWR1,CDR1
0,ACH-000002,0.362532,0.274371,3.343203,1.125238,2.065523,2.576643,0.587594,2.614776,2.558101,...,2.636606,0.382942,2.148868,1.528380,0.671607,0.578567,1.263599,0.468970,0.483255,0.274371
1,ACH-000004,1.302372,0.214211,3.138665,1.800551,2.521042,1.110136,3.039840,2.548120,2.988215,...,2.466102,0.972584,1.917304,0.830707,0.815626,0.779967,1.734648,0.688572,0.639767,0.214211
3,ACH-000006,0.321167,0.272362,3.609158,1.939108,2.653499,2.447577,2.518025,3.351510,2.977455,...,1.918512,0.444875,1.906332,1.028573,0.661440,0.794750,1.778459,1.015335,0.511737,0.272362
4,ACH-000007,1.951300,0.276052,3.463491,2.201603,2.495457,0.500130,0.648881,3.296798,2.965959,...,1.615766,0.492227,2.172098,1.091848,0.481683,0.558021,1.734446,0.534023,0.478507,0.276052
7,ACH-000015,2.414059,0.323905,4.049679,2.050041,2.423360,0.500690,2.349025,3.642805,2.751286,...,2.240014,1.150320,2.227148,2.056217,0.621720,0.691820,2.077332,0.557902,0.509564,0.323905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,ACH-001674,3.041591,0.326950,4.046794,1.332917,2.505366,0.581943,0.596788,0.491734,2.943735,...,2.849016,2.361739,1.552435,1.112233,0.483727,0.714100,1.653016,0.562764,0.474603,0.402230
946,ACH-001702,2.469605,0.363320,3.912677,1.345152,2.314148,0.393528,0.593513,3.878542,2.406774,...,2.628366,2.738316,1.132231,1.379643,1.501885,0.489261,2.040035,0.525732,0.525732,0.476575
948,ACH-001711,2.341874,1.072730,3.867810,1.555023,2.221735,0.469910,3.382413,0.558810,1.907941,...,2.384591,1.480382,2.284725,1.062074,0.551334,0.569685,2.043872,0.775236,0.621449,0.399505
949,ACH-001716,2.702766,0.491362,3.915659,1.333843,2.318513,0.292353,0.492557,2.843101,2.823941,...,3.093360,1.991684,1.059938,0.723125,0.506645,0.490655,1.436047,0.505419,0.505419,0.401076


In [11]:
filter_drug_data_df = drug_viability[drug_viability['cell_line'].isin(common_cell_line)]

In [12]:
filter_drug_data_df

,DRUG_ID,COSMIC_ID,dose,viability,DATASET,cell_line,DRUG_NAME,BROAD_ID
873,1007,1240218,0.012500,0.282956,GDSC1,ACH-000114,DOCETAXEL,"BRD-K30577245, BRD-A05821830, BRD-K63265447, B..."
874,1007,1240218,0.006250,0.361895,GDSC1,ACH-000114,DOCETAXEL,"BRD-K30577245, BRD-A05821830, BRD-K63265447, B..."
875,1007,1240218,0.003125,0.596641,GDSC1,ACH-000114,DOCETAXEL,"BRD-K30577245, BRD-A05821830, BRD-K63265447, B..."
876,1007,1240218,0.001563,0.800204,GDSC1,ACH-000114,DOCETAXEL,"BRD-K30577245, BRD-A05821830, BRD-K63265447, B..."
877,1007,1240218,0.000781,0.969394,GDSC1,ACH-000114,DOCETAXEL,"BRD-K30577245, BRD-A05821830, BRD-K63265447, B..."
...,...,...,...,...,...,...,...,...
4049179,1632,924106,2.000000,0.960885,GDSC2,ACH-000902,RIBOCICLIB,BRD-K36788280
4049180,1632,924106,0.250000,0.988273,GDSC2,ACH-000902,RIBOCICLIB,BRD-K36788280
4049181,1632,924106,0.015625,0.988273,GDSC2,ACH-000902,RIBOCICLIB,BRD-K36788280
4049182,1909,924106,0.062500,0.998547,GDSC2,ACH-000902,VENETOCLAX,BRD-K62391742


In [13]:
filtered_data = filter_drug_data_df[(filter_drug_data_df['DRUG_ID'] == 1006) | (filter_drug_data_df['DRUG_ID'] == 1372) | (filter_drug_data_df['DRUG_ID'] == 1004)
| (filter_drug_data_df['DRUG_ID'] == 1003) | (filter_drug_data_df['DRUG_ID'] == 1032) | (filter_drug_data_df['DRUG_ID'] == 1012) | (filter_drug_data_df['DRUG_ID'] == 1022) |
 (filter_drug_data_df['DRUG_ID'] == 1058) | (filter_drug_data_df['DRUG_ID'] == 1494) | (filter_drug_data_df['DRUG_ID'] == 1057)]

In [14]:
drug_df = filtered_data[['cell_line','DRUG_ID','dose','viability']]

In [15]:
drug_df

,cell_line,DRUG_ID,dose,viability
891,ACH-000114,1032,0.500000,0.765152
892,ACH-000114,1032,0.250000,0.941400
893,ACH-000114,1032,0.125000,0.796375
894,ACH-000114,1032,0.062500,0.916030
895,ACH-000114,1032,0.031250,0.867244
...,...,...,...,...
4046647,ACH-000902,1032,0.039062,0.799816
4046654,ACH-000902,1032,10.000000,0.293989
4046655,ACH-000902,1032,5.000000,0.499927
4046656,ACH-000902,1032,2.500000,0.622763


In [ ]:
# Merge drug and genes data
merge_drug_gene_df = pd.merge(drug_df, result_df_filter, on='cell_line')


# Data Spliting

In [16]:
X = result_df_filter
X_train_cell, X_test_cell = train_test_split(X, test_size=0.07, random_state=42, shuffle=True)  # Adjust test_size as needed

In [17]:
X_train = X_train_cell.drop('cell_line',axis=1)
X_test = X_test_cell.drop('cell_line',axis=1)

In [18]:
X_test

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,DERPC,AC055839.2,AP001453.3,H3C2,H3C3,AC098582.1,DUS4L-BCAP29,C8orf44-SGK3,NPBWR1,CDR1
103,1.434094,0.251808,3.622893,1.282052,1.736237,0.663290,3.366644,3.694026,2.843286,2.941503,...,1.226558,2.256807,2.252891,1.109418,1.062100,0.983834,1.401259,0.993321,0.657052,0.346325
120,2.373379,0.263601,3.834702,1.945505,2.984324,0.678956,3.759623,3.018299,2.297973,3.011774,...,2.406155,2.784232,1.772501,0.754086,0.939178,0.661632,2.296173,0.584845,0.478769,0.284924
65,1.900780,0.299963,3.759663,1.791590,2.677642,0.999750,0.592315,0.648023,3.177042,2.958863,...,2.615074,2.078109,1.100791,0.791664,0.932307,1.022180,1.790247,1.391880,0.947041,0.316201
934,3.041591,0.326950,4.046794,1.332917,2.505366,0.581943,0.596788,0.491734,2.943735,2.812507,...,2.849016,2.361739,1.552435,1.112233,0.483727,0.714100,1.653016,0.562764,0.474603,0.402230
136,1.986485,0.426807,3.939842,1.416796,2.276372,0.609732,0.701070,3.490940,2.111838,2.377541,...,1.787754,3.727078,1.924015,1.155144,1.513006,0.676230,2.052448,0.612183,0.435675,0.419910
404,3.101835,0.600523,3.963858,1.611916,1.638969,0.688529,0.607306,3.490911,3.135042,2.593953,...,2.176882,0.582504,2.300213,1.088505,0.942414,0.590535,1.275458,0.578358,0.543164,0.600523
372,2.207810,0.336844,4.232859,1.987659,2.445673,0.721718,1.883805,3.198861,1.874270,2.669769,...,2.270068,1.804627,2.139295,1.306394,0.969014,0.447486,1.777395,0.850757,0.763418,0.229782
26,0.301141,0.286856,3.496617,1.464299,2.124507,2.345114,0.610767,3.032426,1.607900,2.500483,...,2.169749,1.137925,2.028565,1.250127,1.626163,0.555449,1.170799,0.790565,0.666092,0.286856
306,0.284469,0.263146,3.015648,1.331791,1.823956,0.506887,1.681326,0.619089,2.765961,3.039707,...,1.506522,2.310086,1.939535,1.150240,1.153131,0.663247,1.204837,1.164089,0.865876,0.263959
590,2.003506,0.224985,4.125161,1.638846,2.712905,0.410729,2.974257,3.097924,2.791379,2.264738,...,3.108916,1.118937,1.957581,1.358818,1.599894,0.403417,2.419928,0.589295,0.919001,0.385352


In [20]:
input_dim = X_train.shape[1]
latent_dim = 1000  # Set the size of the latent space

alpha = 0.1
activation = LeakyReLU(alpha=alpha)

input_layer = Input(shape=(input_dim,))  # Assuming input_dim is the dimensionality of your input data

# Encoder layers
encoded = Dense(512, activation=activation)(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.2)(encoded)  # Add dropout layer with dropout rate 0.2
encoded = Dense(128, activation=activation)(encoded)
encoded = BatchNormalization()(encoded)
# encoded = Dropout(0.2)(encoded)  # Add dropout layer with dropout rate 0.2
encoded = Dense(latent_dim, activation=activation)(encoded)  # Latent dimensionality

# Decoder layers
decoded = Dense(128, activation=activation)(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.2)(decoded)  # Add dropout layer with dropout rate 0.2
decoded = Dense(512, activation=activation)(decoded)
decoded = BatchNormalization()(decoded)
# decoded = Dropout(0.2)(decoded)  # Add dropout layer with dropout rate 0.2
decoded = Dense(input_dim, activation=activation)(decoded)

# Define autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile autoencoder
autoencoder.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

# Train the autoencoder
history = autoencoder.fit(X_train, X_train, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/100
13/13 [==============================] - 9s 475ms/step - loss: 3.3130 - val_loss: 14867.2070
Epoch 2/100
13/13 [==============================] - 7s 507ms/step - loss: 1.9159 - val_loss: 58.3873
Epoch 3/100
13/13 [==============================] - 7s 532ms/step - loss: 0.8605 - val_loss: 800.4813
Epoch 4/100
13/13 [==============================] - 5s 393ms/step - loss: 0.5219 - val_loss: 182.9837
Epoch 5/100
13/13 [==============================] - 5s 391ms/step - loss: 0.3921 - val_loss: 54.8856
Epoch 6/100
13/13 [==============================] - 7s 518ms/step - loss: 0.3485 - val_loss: 21.2641
Epoch 7/100
13/13 [==============================] - 5s 366ms/step - loss: 0.3479 - val_loss: 32.3347
Epoch 8/100
13/13 [==============================] - 8s 600ms/step - loss: 0.3211 - val_loss: 10.0182
Epoch 9/100
13/13 [==============================] - 7s 518ms/step - loss: 0.3090 - val_loss: 5.8666
Epoch 10/100
13/13 [==============================] - 9s 677ms/step - loss: 0.

# Feature Extraction

In [22]:
encoder = Model(inputs=input_layer, outputs=encoded)

In [23]:
X_train_features = encoder.predict(X_train) # Train Features
X_test_features = encoder.predict(X_test) # Test Features

2/2 [==============================] - 0s 13ms/step


In [ ]:
# Tran DataFrame with column names
gene_columns = X_train.columns[:1000]
X_train_features_df = pd.DataFrame(X_train_cell.cell_line, columns=['cell_line'])
X_train_features_df[gene_columns] = X_train_features

In [ ]:
# Test DataFrame with column names
gene_columns_test = X_test.columns[:1000]
X_test_features_df = pd.DataFrame(X_test_cell.cell_line, columns=['cell_line'])
X_test_features_df[gene_columns_test] = X_test_features

In [26]:
merge_drug_gene_train_df = pd.merge(drug_df, X_train_features_df, on='cell_line') # Merge drug gene train data
merge_drug_gene_train_df

,cell_line,DRUG_ID,dose,viability,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,...,CACNB1,EVI5,STOML1,PKM,DHX29,DNTTIP2,METTL22,TP53BP1,TRO,RRP15
0,ACH-000114,1032,0.500000,0.765152,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
1,ACH-000114,1032,0.250000,0.941400,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
2,ACH-000114,1032,0.125000,0.796375,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
3,ACH-000114,1032,0.062500,0.916030,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
4,ACH-000114,1032,0.031250,0.867244,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100463,ACH-000902,1032,0.039062,0.799816,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100464,ACH-000902,1032,10.000000,0.293989,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100465,ACH-000902,1032,5.000000,0.499927,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100466,ACH-000902,1032,2.500000,0.622763,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787


In [27]:
merge_drug_gene_test_df = pd.merge(drug_df, X_test_features_df, on='cell_line') # Merge drug gene test data
merge_drug_gene_test_df

,cell_line,DRUG_ID,dose,viability,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,...,CACNB1,EVI5,STOML1,PKM,DHX29,DNTTIP2,METTL22,TP53BP1,TRO,RRP15
0,ACH-000788,1032,0.500000,1.000000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
1,ACH-000788,1032,0.250000,0.931912,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
2,ACH-000788,1032,0.125000,1.000000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
3,ACH-000788,1032,0.062500,0.932842,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
4,ACH-000788,1032,0.031250,1.000000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884,ACH-000944,1372,0.010004,0.931139,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6885,ACH-000944,1372,0.031631,0.932767,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6886,ACH-000944,1372,0.100018,0.948978,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6887,ACH-000944,1372,0.316256,1.000000,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285


In [28]:
encoded_X_train = merge_drug_gene_train_df.drop(['cell_line','viability'],axis=1) # X_train data
encoded_X_train

,DRUG_ID,dose,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,CACNB1,EVI5,STOML1,PKM,DHX29,DNTTIP2,METTL22,TP53BP1,TRO,RRP15
0,1032,0.500000,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,-0.236224,8.698725,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
1,1032,0.250000,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,-0.236224,8.698725,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
2,1032,0.125000,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,-0.236224,8.698725,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
3,1032,0.062500,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,-0.236224,8.698725,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
4,1032,0.031250,-0.271616,-0.066217,-0.518280,-0.821498,2.712940,-0.411152,-0.236224,8.698725,...,-0.299369,-0.031969,0.880418,-0.915885,1.433920,-0.377869,6.271274,-0.507762,-0.018558,-0.416666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100463,1032,0.039062,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,-0.567553,-0.209874,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100464,1032,10.000000,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,-0.567553,-0.209874,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100465,1032,5.000000,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,-0.567553,-0.209874,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787
100466,1032,2.500000,5.393842,-0.336358,-0.437384,2.003122,0.000845,-0.290737,-0.567553,-0.209874,...,-0.406263,-0.315372,-0.585346,-0.771732,-0.305382,-0.310470,6.617570,-0.220361,-0.686208,-0.452787


In [29]:
encoded_Y_train = merge_drug_gene_train_df['viability'] # Y_train Data
encoded_Y_train

0         0.765152
1         0.941400
2         0.796375
3         0.916030
4         0.867244
            ...   
100463    0.799816
100464    0.293989
100465    0.499927
100466    0.622763
100467    0.991312
Name: viability, Length: 100468, dtype: float64

In [30]:
encoded_X_test = merge_drug_gene_test_df.drop(['cell_line','viability'],axis=1) # X_test data
encoded_X_test

,DRUG_ID,dose,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,CACNB1,EVI5,STOML1,PKM,DHX29,DNTTIP2,METTL22,TP53BP1,TRO,RRP15
0,1032,0.500000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,0.059413,-0.232518,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
1,1032,0.250000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,0.059413,-0.232518,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
2,1032,0.125000,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,0.059413,-0.232518,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
3,1032,0.062500,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,0.059413,-0.232518,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
4,1032,0.031250,4.628846,-0.139062,-0.348863,1.831792,-0.130975,-0.445878,0.059413,-0.232518,...,-0.128365,1.100072,-0.148299,7.108926,-0.197189,-0.068549,-0.724566,2.761754,8.054280,-0.345953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6884,1372,0.010004,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,-0.228567,2.756106,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6885,1372,0.031631,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,-0.228567,2.756106,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6886,1372,0.100018,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,-0.228567,2.756106,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285
6887,1372,0.316256,6.525533,-0.246755,-0.312727,3.717928,1.445748,6.408624,-0.228567,2.756106,...,-0.246552,0.308015,3.472074,5.853337,4.040447,3.375259,-0.218491,1.152454,-0.211349,1.991285


In [31]:
encoded_Y_test = merge_drug_gene_test_df['viability'] # Y_test data
encoded_Y_test

0       1.000000
1       0.931912
2       1.000000
3       0.932842
4       1.000000
          ...   
6884    0.931139
6885    0.932767
6886    0.948978
6887    1.000000
6888    0.950589
Name: viability, Length: 6889, dtype: float64

In [32]:
# DataFrame Values
np_arr = encoded_X_train.values

# DNN

In [33]:
model = tf.keras.Sequential([
    Dense(128, input_shape=(np_arr.shape[1],)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),  # Adding dropout with 20% dropout rate after the first hidden layer
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),  # Adding dropout with 20% dropout rate after the second hidden layer
    # Dense(32),     # Adding an additional hidden layer
    # BatchNormalization(),
    # Activation('relu'),
    # Dropout(0.1),  # Adding dropout with 20% dropout rate after the third hidden layer
    Dense(1)       # Output layer for regression, with 1 neuron
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
history = model.fit(np_arr, encoded_Y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=2)

Epoch 1/20
2826/2826 - 17s - loss: 0.0748 - val_loss: 0.0949 - 17s/epoch - 6ms/step
Epoch 2/20
2826/2826 - 17s - loss: 0.0638 - val_loss: 0.1012 - 17s/epoch - 6ms/step
Epoch 3/20
2826/2826 - 14s - loss: 0.0633 - val_loss: 0.0937 - 14s/epoch - 5ms/step
Epoch 4/20
2826/2826 - 14s - loss: 0.0625 - val_loss: 0.0885 - 14s/epoch - 5ms/step
Epoch 5/20
2826/2826 - 14s - loss: 0.0618 - val_loss: 0.0966 - 14s/epoch - 5ms/step
Epoch 6/20
2826/2826 - 14s - loss: 0.0612 - val_loss: 0.0933 - 14s/epoch - 5ms/step
Epoch 7/20
2826/2826 - 14s - loss: 0.0609 - val_loss: 0.0900 - 14s/epoch - 5ms/step
Epoch 8/20
2826/2826 - 14s - loss: 0.0603 - val_loss: 0.0892 - 14s/epoch - 5ms/step
Epoch 9/20
2826/2826 - 14s - loss: 0.0602 - val_loss: 0.0874 - 14s/epoch - 5ms/step
Epoch 10/20
2826/2826 - 15s - loss: 0.0596 - val_loss: 0.0823 - 15s/epoch - 5ms/step
Epoch 11/20
2826/2826 - 16s - loss: 0.0596 - val_loss: 0.0890 - 16s/epoch - 6ms/step
Epoch 12/20
2826/2826 - 15s - loss: 0.0595 - val_loss: 0.0927 - 15s/epoch 